In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier


In [3]:
url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
tcc_df = pd.read_csv(url)

print("Shape do dataset:", tcc_df.shape)
display(tcc_df.head())
print("\nInformações do dataset:")
tcc_df.info()

print("\nDistribuição da variável alvo (Churn):")
tcc_df['Churn'].value_counts(normalize=True)


Shape do dataset: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes



Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling 

,proportion
Churn,
No,0.73463
Yes,0.26537


In [4]:
tcc_df.replace(" ", np.nan, inplace=True)

missing = tcc_df.isnull().sum()
print("\nDados faltantes por coluna:")
print(missing[missing > 0])

tcc_df.dropna(inplace=True)
print("\nShape apos remoção de dados faltantes:", tcc_df.shape)


Dados faltantes por coluna:
TotalCharges    11
dtype: int64

Shape após remoção de dados faltantes: (7032, 21)


In [7]:
cat_cols = tcc_df.select_dtypes(include=['object']).columns.tolist()
cat_cols.remove('customerID')
cat_cols.remove('Churn')
le = LabelEncoder()
for col in cat_cols:
    tcc_df[col] = le.fit_transform(tcc_df[col])
tcc_df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,2504,No
1,5575-GNVDE,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1465,No
2,3668-QPYBK,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,156,Yes
3,7795-CFOCW,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1399,No
4,9237-HQITU,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,924,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,24,1,2,0,2,...,2,2,2,2,1,1,3,84.80,1596,No
7039,2234-XADUH,0,0,1,1,72,1,2,1,0,...,2,0,2,2,1,1,1,103.20,5697,No
7040,4801-JZAZL,0,0,1,1,11,0,1,0,2,...,0,0,0,0,0,1,2,29.60,2993,No
7041,8361-LTMKD,1,1,1,0,4,1,2,1,0,...,0,0,0,0,0,1,3,74.40,2659,Yes


In [8]:
X = tcc_df.drop(['customerID', 'Churn'], axis=1)
y = tcc_df['Churn'].map({'No': 0, 'Yes': 1})
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

Shape de X: (7032, 19)
Shape de y: (7032,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print(f"Tamanho treino: {X_train.shape[0]} amostras")
print(f"Tamanho teste: {X_test.shape[0]} amostras")

Tamanho treino: 4922 amostras
Tamanho teste: 2110 amostras


In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [12]:
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.89      0.87      1549
           1       0.65      0.54      0.59       561

    accuracy                           0.80      2110
   macro avg       0.74      0.72      0.73      2110
weighted avg       0.79      0.80      0.79      2110

Confusion Matrix:
 [[1384  165]
 [ 260  301]]
ROC AUC Score: 0.8345


In [13]:
# Predição das probabilidades da classe positiva
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# Ajusta threshold para 0.3, por exemplo
threshold = 0.3
y_pred_adj = np.where(y_proba >= threshold, 1, 0)
print(classification_report(y_test, y_pred_adj))

              precision    recall  f1-score   support

           0       0.89      0.74      0.81      1549
           1       0.52      0.76      0.62       561

    accuracy                           0.75      2110
   macro avg       0.71      0.75      0.71      2110
weighted avg       0.79      0.75      0.76      2110



In [14]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

print("Shape após SMOTE:", X_train_res.shape, y_train_res.shape)

# Treina o modelo com dados balanceados
model_bal = LogisticRegression(max_iter=1000, random_state=42)
model_bal.fit(X_train_res, y_train_res)

# Avalia com threshold padrão
y_pred_bal = model_bal.predict(X_test_scaled)
y_proba_bal = model_bal.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred_bal))

Shape após SMOTE: (7228, 19) (7228,)
              precision    recall  f1-score   support

           0       0.90      0.73      0.80      1549
           1       0.51      0.77      0.61       561

    accuracy                           0.74      2110
   macro avg       0.70      0.75      0.71      2110
weighted avg       0.79      0.74      0.75      2110



In [17]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)

y_pred_rf = rf.predict(X_test_scaled)
y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1549
           1       0.60      0.46      0.52       561

    accuracy                           0.77      2110
   macro avg       0.71      0.67      0.69      2110
weighted avg       0.76      0.77      0.76      2110

